In [ ]:
# importações das bibliotecas necessárias
import pandas as pd
import re
from datetime import datetime, date
import matplotlib.pyplot as plt
import numpy as np
from numpy import mean

# Importação dos dados
healthcare_df = pd.read_csv('/content/drive/MyDrive/healthcare_dataset.csv')
df_clean = healthcare_df.copy()  # fazemos uma cópia para preservar o original

# Definição de funções de tratamento de dados

def tratar_dados_ausentes(data):
    # converte “Room Number” para numérico (coerce transforma valores inválidos em NaN)
    data["Room Number"] = pd.to_numeric(data["Room Number"], errors='coerce')
    # converte “Billing Amount” para numérico
    data["Billing Amount"] = pd.to_numeric(data["Billing Amount"], errors='coerce')
    # converte “Age” para numérico
    data["Age"] = pd.to_numeric(data["Age"], errors='coerce')

    # se houver colunas do tipo object com valores nulos
    if data.select_dtypes(include=['object']).isnull().values.any():
        # preenche com “Valor não definido”
        data.fillna("Valor não definido", inplace=True)
    return data

def tratar_fatura_negativa(data):
    # para todas as faturas menores que zero, definir como NaN
    data.loc[data['Billing Amount'] < 0, 'Billing Amount'] = np.nan
    # se houver NaNs em “Billing Amount”
    if data['Billing Amount'].isnull().values.any():
        # preencher os NaNs pela média da coluna
        data.fillna(data['Billing Amount'].mean(), inplace=True)
    return data

def tratar_data(data):
    # converte “Date of Admission” para datetime (coerce para valores inválidos virarem NaT)
    data["Date of Admission"] = pd.to_datetime(data['Date of Admission'], errors='coerce')
    # converte “Discharge Date” para datetime
    data["Discharge Date"] = pd.to_datetime(data['Discharge Date'], errors='coerce')
    # preenche datas faltantes com data atual formatada
    data["Date of Admission"].fillna({"Date of Admission": data_formatada})
    data["Discharge Date"].fillna({"Discharge Date": data_formatada})
    return data

def tratar_nomes(data):
    # para todas as colunas tipo object, aplicar title case (Primeira letra maiúscula)
    data[data.select_dtypes(include=['object']).columns] = \
        data.select_dtypes(include=['object']).apply(lambda x: x.str.title())
    return data

def identificar_outliers(data):
    # calcula Q1 e Q3
    Q1 = data.quantile(0.25)
    Q3 = data.quantile(0.75)
    IQR = Q3 - Q1
    # define limites inferior e superior
    limite_inferior = Q1 - 1.5 * IQR
    limite_superior = Q3 + 1.5 * IQR
    # retorna os valores que são outliers
    outliers = data[(data < limite_inferior) | (data > limite_superior)]
    return outliers

def normalizar_dado(data):
    # normaliza a série: (valor - mínimo) / (máximo - mínimo)
    data = (data - data.min()) / (data.max() - data.min())
    return data

# Aplicação das funções de tratamento no DataFrame limpo
agora = datetime.now()
data_formatada = agora.strftime("%Y-%m-%d")

tratar_data(df_clean)
tratar_nomes(df_clean)
tratar_dados_ausentes(df_clean)
tratar_fatura_negativa(df_clean)

# Criação de nova coluna “Days in Hospital” com número de dias internado
df_clean['Days in Hospital'] = df_clean['Discharge Date'] - df_clean['Date of Admission']
df_clean['Days in Hospital'] = df_clean['Days in Hospital'].dt.days  # extrai apenas os dias

# Normalização da coluna de fatura
df_clean['Billing Normalized'] = normalizar_dado(df_clean['Billing Amount'])

# Cálculo da média de “Billing Normalized” por seguradora
media_preco_por_seguradora = df_clean.groupby('Insurance Provider').agg({'Billing Normalized': 'mean'})

# Criação de faixa etária (“Age Bracket”) para facilitar segmentações
df_clean['Age Bracket'] = pd.cut(df_clean['Age'],
                                 bins=[12, 18, 40, 65, 100],
                                 labels=['13-18', '18-40', '40-65', '65+'],
                                 right=False)

# Média de custo por condição médica
media_custo_por_condicao = df_clean.groupby('Medical Condition').agg({'Billing Normalized':'mean'})

# Frequência relativa de doenças por faixa etária (%)
frequencia_doencas_por_idade = df_clean['Age Bracket'].groupby(df_clean['Medical Condition']).value_counts(normalize=True) * 100

# Gráfico: custo médio por faixa etária
media_billing_por_idade = df_clean.groupby('Age Bracket')['Billing Amount'].mean()
plt.figure(figsize=(10, 6))
plt.bar(media_billing_por_idade.index.get_level_values(0),
        media_billing_por_idade.values,
        label=media_billing_por_idade.index,
        color=['#33C3FF', '#75FF33', '#FFC300', '#FF5733', '#FF8230'],
        width=0.5)
plt.legend()
plt.xlabel('Faixa Etária')
plt.ylabel('Custo Médio (Billing Amount)')
plt.title('Custo Médio por Faixa Etária')
plt.show()

# Gráfico: frequência de medicação por condição médica
medicacao_por_condicao = df_clean.groupby('Medication')['Medical Condition'].value_counts(normalize=True) * 100
medicacao_por_condicao.unstack().plot(kind='bar',
                                      color=['#33C3FF', '#75FF33', '#FFC300', '#FF5733'],
                                      stacked=True)
plt.xlabel('Medicação')
plt.ylabel('Frequência Relativa (%)')
plt.title('Frequência de Medicação por Condição')
plt.xticks(rotation=45)
plt.show()

# Gráfico: custo médio por seguradora
custo_medio_por_seguradora = df_clean.groupby('Insurance Provider')['Billing Amount'].mean()
plt.figure(figsize=(10, 6))
plt.bar(custo_medio_por_seguradora.index,
        custo_medio_por_seguradora.values,
        width=0.5,
        label=custo_medio_por_seguradora.index,
        color=['#33C3FF', '#75FF33', '#FFC300', '#FF5733', '#FF8230'])
plt.legend()
plt.xlabel('Seguradora')
plt.ylabel('Custo Médio (Billing Amount)')
plt.title('Custo Médio por Seguradora')
plt.xticks(rotation=45)
plt.show()

# Gráfico: proporção de pessoas por seguradora
frequencia_pessoas_cadastradas_na_seguradora = df_clean['Insurance Provider'].value_counts(normalize=True) * 100
plt.pie(frequencia_pessoas_cadastradas_na_seguradora,
        labels=frequencia_pessoas_cadastradas_na_seguradora.index,
        autopct='%1.1f%%')
plt.axis('equal')
plt.title('Frequência de Pessoas Cadastradas por Seguradora')
plt.show()

# Gráficos: distribuição por gênero e por seguradora
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 6))

frequencia_por_genero = df_clean['Gender'].value_counts(normalize=True) * 100
ax1.bar(frequencia_por_genero.index,
        frequencia_por_genero.values,
        color=['#33C3FF', '#ffcbdb'],
        width=0.5,
        label=frequencia_por_genero.index)
ax1.legend()
ax1.set_xlabel('Gênero')
ax1.set_ylabel('Frequência Relativa (%)')
ax1.set_title('Frequência de Gênero')

quantidade_por_genero_seguradora = (df_clean
    .groupby('Insurance Provider')['Gender']
    .value_counts(normalize=True) * 100)
df2 = quantidade_por_genero_seguradora.unstack().fillna(0)
df2.plot(kind='bar',
         ax=ax2,
         color=['#33C3FF', '#ffcbdb'],
         stacked=True)
ax2.set_xlabel('Seguradora')
ax2.set_ylabel('Frequência Relativa (%)')
ax2.set_title('Quantidade de Pessoas por Gênero e Seguradora')
ax2.set_xticklabels(ax2.get_xticklabels(), rotation=45, ha='right')

plt.tight_layout()
plt.show()

# Gráfico: preço médio por gênero (boxplot + média)
grupo_m = df_clean.loc[df_clean['Gender']=='Male', 'Billing Amount']
grupo_f = df_clean.loc[df_clean['Gender']=='Female', 'Billing Amount']
medias = [grupo_m.mean(), grupo_f.mean()]

plt.boxplot([grupo_m, grupo_f], labels=['Male', 'Female'])
plt.scatter([1, 2], medias, color='red', marker='D', label='Média')
plt.ylabel('Preço Médio (Billing Amount)')
plt.title('Preço Médio por Gênero')
plt.legend()
plt.show()

# Gráfico: tipo de admissão por condição médica
admissao_por_condicao_medica = df_clean.groupby('Medical Condition')['Admission Type'].value_counts(normalize=True)
admissao_por_condicao_medica.unstack().plot(kind='bar',
                                            color=['#33C3FF', '#75FF33', '#FF5733'],
                                            stacked=True)
plt.xlabel('Condição Médica')
plt.ylabel('Frequência Relativa (%)')
plt.title('Admissão por Condição Médica')
plt.xticks(rotation=45)
plt.show()

# Gráfico: resultados de testes por condição médica
resultado_testes = df_clean['Test Results'].value_counts()
resultado_testes_por_condicao = df_clean.groupby('Medical Condition')['Test Results'].value_counts(normalize=True) * 100

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 6))
ax1.pie(resultado_testes, labels=resultado_testes.index, autopct='%1.1f%%')
ax1.axis('equal')

resultado_testes_por_condicao.unstack().plot(kind='bar',
                                             ax=ax2,
                                             color=['#33C3FF', '#75FF33', '#FF5733'],
                                             stacked=True)
ax2.set_xlabel('Condição Médica')
ax2.set_ylabel('Frequência Relativa (%)')
ax2.set_title('Resultado dos Testes por Condição Médica')
ax2.set_xticklabels(ax2.get_xticklabels(), rotation=45, ha='right')

plt.tight_layout()
plt.show()

# Gráfico: relação entre tempo de internação e preço
relacao_tempo_de_internacao_com_preco = df_clean.groupby('Days in Hospital')['Billing Amount'].mean()

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 6))
ax1.boxplot([relacao_tempo_de_internacao_com_preco])
ax1.set_ylabel('Preço Médio (Billing Amount)')
ax1.set_title('Relação entre Tempo de Internação e Preço Médio')

ax2.scatter(relacao_tempo_de_internacao_com_preco.index,
            relacao_tempo_de_internacao_com_preco.values,
            color='red', marker='o', label='Média', s=10)
ax2.set_xlabel('Tempo de Internação (Dias)')
ax2.set_ylabel('Preço Médio (Billing Amount)')
ax2.set_title('Relação entre Tempo de Internação e Preço Médio')
ax2.legend()

plt.tight_layout()
plt.show()

# Criação de nova coluna com categorias de “Hospitais Length of Stay”
df_clean['Hospital Length of Stay'] = df_clean['Days in Hospital'].apply(
    lambda days: '1-15' if days <= 15 else ('16-30' if days <= 30 else '30+')
)
relacao_tempo_de_internacao_por_condicao = df_clean.groupby('Medical Condition')['Hospital Length of Stay'].value_counts()

relacao_tempo_de_internacao_por_condicao.unstack().plot(kind='bar',
                                                       color=['#33C3FF', '#FF5733'],
                                                       stacked=True)
plt.xlabel('Condição Médica')
plt.ylabel('Quantidade')
plt.title('Tempo de Internação por Condição Médica')
plt.xticks(rotation=45)
plt.show()
